# PyTorch QuickStart

`fashionMNIST` 内のオブジェクトを分類する識別モデルを作成する. https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

## ライブラリの読み込み
ついでにCUDAが使えるかどうかについても確認する．

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"We uses {device} in this container")

## データセットの読み込み
前述したように， `fashionMNIST` 内のオブジェクトを分類する識別モデルを作成する．

In [ ]:
train_data = datasets.FashionMNIST(
    root="dataset",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="dataset",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
BATCHSIZE=64

train_dataloader = DataLoader(train_data, BATCHSIZE)
test_dataloader = DataLoader(test_data, BATCHSIZE)

for x, y in test_dataloader:
    print(f"\rShape of x [N, C, H, W]: {x.shape}, y: {y.shape}({y.dtype})", end="")

## モデルの作成
`Getting Started` ではシンプルなニューラルネットワークで構成されている．

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.sequence = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(),
            nn.Linear(512, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Linear(128, 10),
        )
        
    def forward(self, x):
        res = self.flatten(x)
        res = self.sequence(res)
        return res
    
model = NeuralNetwork().to(device)

In [ ]:
print(model)

## 学習とテスト
損失関数は交差エントロピー誤差，オプティマイザはAdamを使用する．

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
def train(dataloader, model, loss_fn, optimizer, i_epoch):
    size = len(dataloader.dataset)
    model.train()
    for i_batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)
        
        # prediction error
        pred = model(x)
        loss = loss_fn(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i_batch % 100 == 0:
            loss, current = loss.item(), i_batch * len(x)
            print(f"\r{i_epoch} epoch (loss: {loss:>7f}) [{current:>5d}/{size:>5d}]",end="")

def test(dataloader, model, loss_fn, i_epoch):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for x, y in dataloader:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"{i_epoch} test accuracy: {100*correct:0.1f}, avg loss: {test_loss:>8f}")
        

In [ ]:
NUMEPOCHS = 5
for i_epoch in range(NUMEPOCHS):
    train(train_dataloader, model, loss_func, optimizer, i_epoch)
    print("")
    test(test_dataloader, model, loss_func, i_epoch)


## パラメータの保存

In [ ]:
torch.save(model.state_dict(), "trained/model.pth")

In [ ]:
dic = model.state_dict()
for key, tensor in dic.items():
    print(f"{key} parameter's shape: {tensor.shape}")

In [ ]:
del model

In [ ]:
torch.cuda.empty_cache()